# Air Quality Modeling - All-in-One Notebook

This notebook runs end-to-end in one file with no local module imports.
It loads OpenAQ data, cleans it, computes AQI, engineers features, trains models,
evaluates performance, and writes a short report.


## A. Introduction
Air quality reflects the concentration of pollutants in the air and their impacts on health.
The Air Quality Index (AQI) converts pollutant concentrations into a standardized scale for interpretation.

OpenAQ provides raw pollutant concentrations but not AQI because AQI depends on chosen standards,
averaging windows, and unit conversions. We compute AQI explicitly in this notebook.


## B. Data Loading
We load the local CSV and inspect schema and samples.

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reproducibility
SEED = 42
np.random.seed(SEED)

ROOT = Path('..').resolve()
data_path = ROOT / 'data' / 'openaq.csv'

def load_raw_data(path: str) -> pd.DataFrame:
    # OpenAQ exports often use semicolon delimiters; allow fallback sniffing.
    try:
        df = pd.read_csv(path, sep=';', encoding='utf-8')
        if df.shape[1] <= 1:
            raise ValueError('Unexpected delimiter; falling back to sniffing.')
    except Exception:
        df = pd.read_csv(path, sep=None, engine='python', encoding='utf-8')
    return df

raw_df = load_raw_data(str(data_path))
print(raw_df.shape)
display(raw_df.head())
display(raw_df.dtypes)


## C. Data Quality Checks
We check missing values, duplicates, unit inconsistencies, and outline an outlier detection plan.

In [ ]:
# Missing values
missing_pct = raw_df.isna().mean().sort_values(ascending=False)
display(missing_pct.to_frame('missing_pct'))

# Duplicates
dup_count = raw_df.duplicated().sum()
print(f'Duplicate rows: {dup_count}')

# Unit inconsistencies per pollutant
unit_counts = (raw_df[['Pollutant', 'Unit']].dropna().value_counts().reset_index())
unit_counts.columns = ['Pollutant', 'Unit', 'count']
display(unit_counts.head(20))

# Plot: missing value percentages
plt.figure(figsize=(8, 4))
missing_pct.plot(kind='bar')
plt.title('Missing Value Percentage by Column')
plt.ylabel('Percentage')
plt.tight_layout()
plt.show()


**Outlier detection plan**
- Use robust statistics (IQR/median absolute deviation) per pollutant.
- Compare extreme values to AQI breakpoints to flag implausible spikes.
- Review outliers per location to distinguish sensor faults vs real events.


## D. Data Engineering: Long ? Wide Transformation

In [ ]:
import re

POLLUTANTS = {'pm25', 'pm10', 'no2', 'o3', 'co', 'so2'}

def normalize_unit(unit):
    if unit is None:
        return None
    unit = str(unit).strip().lower()
    unit = unit.replace('\u00b5', 'u')
    unit = unit.replace('ug/m^3', 'ug/m3')
    unit = unit.replace('ug/m\u00b3', 'ug/m3')
    unit = unit.replace('mg/m\u00b3', 'mg/m3')
    return unit

def parse_coordinates(value):
    if value is None or (isinstance(value, float) and np.isnan(value)):
        return None, None
    text = str(value)
    numbers = re.findall(r'-?\d+\.\d+|-?\d+', text)
    if len(numbers) >= 2:
        return float(numbers[0]), float(numbers[1])
    return None, None

def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    rename_map = {}
    for col in df.columns:
        norm = col.strip().lower().replace(' ', '_').replace('-', '_')
        if norm in ('country', 'city', 'location', 'coordinates', 'pollutant', 'value'):
            rename_map[col] = norm
        elif norm in ('unit',):
            rename_map[col] = 'unit'
        elif norm in ('source_name', 'source'):
            rename_map[col] = 'source_name'
        elif norm in ('last_updated', 'last_updated_utc', 'datetime'):
            rename_map[col] = 'last_updated'
    return df.rename(columns=rename_map)

df = standardize_columns(raw_df)
df['pollutant'] = df['pollutant'].astype(str).str.strip().str.lower()
df = df[df['pollutant'].isin(POLLUTANTS)]
df['unit'] = df['unit'].apply(normalize_unit)
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df['timestamp'] = pd.to_datetime(df['last_updated'], errors='coerce', utc=True).dt.tz_convert(None)

lat_lon = df['coordinates'].apply(parse_coordinates)
df['latitude'] = lat_lon.apply(lambda x: x[0])
df['longitude'] = lat_lon.apply(lambda x: x[1])

# Save cleaned long data (without AQI yet)
processed_long_path = ROOT / 'data' / 'processed_long.csv'
df.to_csv(processed_long_path, index=False)

# Aggregate daily to reduce sparsity
df['timestamp'] = df['timestamp'].dt.floor('D')
group_cols = [c for c in ['country','city','location','latitude','longitude','timestamp','pollutant'] if c in df.columns]
grouped = df.groupby(group_cols, dropna=False)['value'].mean().reset_index()

wide_df = grouped.pivot_table(
    index=[c for c in ['country','city','location','latitude','longitude','timestamp'] if c in grouped.columns],
    columns='pollutant',
    values='value',
    aggfunc='mean',
).reset_index()
wide_df.columns.name = None

processed_wide_path = ROOT / 'data' / 'processed_wide.csv'
wide_df.to_csv(processed_wide_path, index=False)

display(wide_df.head())

# Plot a sample time series
sample_location = wide_df['location'].iloc[0]
sample = wide_df[wide_df['location'] == sample_location].sort_values('timestamp')
pollutant_for_plot = 'pm25' if 'pm25' in sample.columns else ('pm10' if 'pm10' in sample.columns else sample.select_dtypes('number').columns[-1])
plt.figure(figsize=(10, 4))
plt.plot(sample['timestamp'], sample[pollutant_for_plot], label=pollutant_for_plot.upper())
plt.title(f'{pollutant_for_plot.upper()} Trend - {sample_location}')
plt.xlabel('Date')
plt.ylabel(f'{pollutant_for_plot.upper()} (standardized units)')
plt.legend()
plt.tight_layout()
plt.show()


## E. AQI Computation
We use US EPA AQI breakpoints.

IAQI = (Ihi - Ilo) / (BPhi - BPlo) * (C - BPlo) + Ilo


In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Breakpoint:
    bp_low: float
    bp_high: float
    i_low: int
    i_high: int

BREAKPOINTS = {
    'pm25': {
        'unit': 'ug/m3',
        'table': [
            Breakpoint(0.0, 12.0, 0, 50),
            Breakpoint(12.1, 35.4, 51, 100),
            Breakpoint(35.5, 55.4, 101, 150),
            Breakpoint(55.5, 150.4, 151, 200),
            Breakpoint(150.5, 250.4, 201, 300),
            Breakpoint(250.5, 350.4, 301, 400),
            Breakpoint(350.5, 500.4, 401, 500),
        ],
    },
    'pm10': {
        'unit': 'ug/m3',
        'table': [
            Breakpoint(0, 54, 0, 50),
            Breakpoint(55, 154, 51, 100),
            Breakpoint(155, 254, 101, 150),
            Breakpoint(255, 354, 151, 200),
            Breakpoint(355, 424, 201, 300),
            Breakpoint(425, 504, 301, 400),
            Breakpoint(505, 604, 401, 500),
        ],
    },
    'o3': {
        'unit': 'ppm',
        'table': [
            Breakpoint(0.000, 0.054, 0, 50),
            Breakpoint(0.055, 0.070, 51, 100),
            Breakpoint(0.071, 0.085, 101, 150),
            Breakpoint(0.086, 0.105, 151, 200),
            Breakpoint(0.106, 0.200, 201, 300),
            Breakpoint(0.201, 0.604, 301, 500),
        ],
    },
    'co': {
        'unit': 'ppm',
        'table': [
            Breakpoint(0.0, 4.4, 0, 50),
            Breakpoint(4.5, 9.4, 51, 100),
            Breakpoint(9.5, 12.4, 101, 150),
            Breakpoint(12.5, 15.4, 151, 200),
            Breakpoint(15.5, 30.4, 201, 300),
            Breakpoint(30.5, 40.4, 301, 400),
            Breakpoint(40.5, 50.4, 401, 500),
        ],
    },
    'so2': {
        'unit': 'ppb',
        'table': [
            Breakpoint(0, 35, 0, 50),
            Breakpoint(36, 75, 51, 100),
            Breakpoint(76, 185, 101, 150),
            Breakpoint(186, 304, 151, 200),
            Breakpoint(305, 604, 201, 300),
            Breakpoint(605, 804, 301, 400),
            Breakpoint(805, 1004, 401, 500),
        ],
    },
    'no2': {
        'unit': 'ppb',
        'table': [
            Breakpoint(0, 53, 0, 50),
            Breakpoint(54, 100, 51, 100),
            Breakpoint(101, 360, 101, 150),
            Breakpoint(361, 649, 151, 200),
            Breakpoint(650, 1249, 201, 300),
            Breakpoint(1250, 1649, 301, 400),
            Breakpoint(1650, 2049, 401, 500),
        ],
    },
}

MOLECULAR_WEIGHTS = {
    'o3': 48.00,
    'no2': 46.01,
    'so2': 64.07,
    'co': 28.01,
}

def ugm3_to_ppm(value_ugm3: float, mw: float) -> float:
    return (value_ugm3 * 24.45) / (mw * 1000.0)

def normalize_unit(unit):
    if unit is None:
        return None
    unit = str(unit).strip().lower()
    unit = unit.replace('\u00b5', 'u')
    unit = unit.replace('ug/m^3', 'ug/m3')
    unit = unit.replace('ug/m\u00b3', 'ug/m3')
    unit = unit.replace('mg/m\u00b3', 'mg/m3')
    return unit

def convert_to_standard(pollutant, value, unit):
    if value is None or (isinstance(value, float) and np.isnan(value)):
        return None, None
    if pollutant not in BREAKPOINTS:
        return None, None
    unit = normalize_unit(unit)
    target_unit = BREAKPOINTS[pollutant]['unit']
    if unit == target_unit:
        return float(value), target_unit
    if pollutant in ('pm25','pm10'):
        if unit == 'mg/m3':
            return float(value) * 1000.0, target_unit
        if unit == 'ug/m3':
            return float(value), target_unit
        return None, None
    mw = MOLECULAR_WEIGHTS.get(pollutant)
    if mw is None:
        return None, None
    if target_unit == 'ppm':
        if unit == 'ppb':
            return float(value) / 1000.0, target_unit
        if unit == 'ug/m3':
            return ugm3_to_ppm(float(value), mw), target_unit
        if unit == 'mg/m3':
            return ugm3_to_ppm(float(value) * 1000.0, mw), target_unit
        if unit == 'ppm':
            return float(value), target_unit
    if target_unit == 'ppb':
        if unit == 'ppm':
            return float(value) * 1000.0, target_unit
        if unit == 'ug/m3':
            return ugm3_to_ppm(float(value), mw) * 1000.0, target_unit
        if unit == 'mg/m3':
            return ugm3_to_ppm(float(value) * 1000.0, mw) * 1000.0, target_unit
        if unit == 'ppb':
            return float(value), target_unit
    return None, None

def compute_iaqi(pollutant, concentration, unit=None):
    if pollutant not in BREAKPOINTS:
        return None
    if unit is None:
        unit = BREAKPOINTS[pollutant]['unit']
    converted, _ = convert_to_standard(pollutant, concentration, unit)
    if converted is None:
        return None
    for bp in BREAKPOINTS[pollutant]['table']:
        if bp.bp_low <= converted <= bp.bp_high:
            return ((bp.i_high - bp.i_low) / (bp.bp_high - bp.bp_low)) * (converted - bp.bp_low) + bp.i_low
    return None

def aqi_category(aqi):
    if aqi is None or (isinstance(aqi, float) and np.isnan(aqi)):
        return None
    if aqi <= 50:
        return 'Good'
    if aqi <= 100:
        return 'Moderate'
    if aqi <= 150:
        return 'Unhealthy for Sensitive Groups'
    if aqi <= 200:
        return 'Unhealthy'
    if aqi <= 300:
        return 'Very Unhealthy'
    if aqi <= 500:
        return 'Hazardous'
    return 'Hazardous'

def compute_aqi_dataframe(df):
    aqi_values = []
    for _, row in df.iterrows():
        iaqis = []
        for p in BREAKPOINTS.keys():
            if p in row:
                iaqi = compute_iaqi(p, row[p])
                if iaqi is not None:
                    iaqis.append(iaqi)
        aqi_values.append(float(np.max(iaqis)) if iaqis else None)
    out = df.copy()
    out['aqi'] = aqi_values
    out['aqi_category'] = out['aqi'].apply(aqi_category)
    return out

aqi_df = compute_aqi_dataframe(wide_df)
processed_aqi_path = ROOT / 'data' / 'processed_aqi.csv'
aqi_df.to_csv(processed_aqi_path, index=False)

plt.figure(figsize=(8, 4))
aqi_df['aqi'].dropna().hist(bins=30)
plt.title('AQI Distribution')
plt.xlabel('AQI')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()


## F. Feature Engineering

In [ ]:
def add_time_features(df, time_col='timestamp'):
    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col], errors='coerce')
    df['hour'] = df[time_col].dt.hour
    df['day_of_week'] = df[time_col].dt.dayofweek
    df['month'] = df[time_col].dt.month
    return df

def add_lag_features(df, group_cols, target_cols, lags=(1,), time_col='timestamp'):
    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col], errors='coerce')
    df = df.sort_values(list(group_cols) + [time_col])
    for lag in lags:
        for col in target_cols:
            if col not in df.columns:
                continue
            lag_col = f'{col}_lag{lag}'
            df[lag_col] = df.groupby(list(group_cols))[col].shift(lag)
    return df

pollutant_cols = ['pm25', 'pm10', 'no2', 'o3', 'co', 'so2']
features_df = add_time_features(aqi_df, time_col='timestamp')
features_df = add_lag_features(features_df, group_cols=['location'], target_cols=pollutant_cols, lags=[1])

processed_features_path = ROOT / 'data' / 'processed_features.csv'
features_df.to_csv(processed_features_path, index=False)

numeric_cols = pollutant_cols + ['aqi', 'hour', 'day_of_week', 'month', 'latitude', 'longitude']
numeric_cols = [c for c in numeric_cols if c in features_df.columns]
corr = features_df[numeric_cols].corr()
plt.figure(figsize=(8, 6))
plt.imshow(corr, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar()
plt.xticks(range(len(numeric_cols)), numeric_cols, rotation=45, ha='right')
plt.yticks(range(len(numeric_cols)), numeric_cols)
plt.title('Feature Correlation Heatmap')
plt.tight_layout()
plt.show()


## G. Modeling

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def train_test_split_time(df, time_col='timestamp', test_size=0.2):
    df = df.sort_values(time_col)
    split_index = int(len(df) * (1 - test_size))
    return df.iloc[:split_index].copy(), df.iloc[split_index:].copy()

def regression_metrics(y_true, y_pred):
    return {
        'mae': mean_absolute_error(y_true, y_pred),
        'rmse': mean_squared_error(y_true, y_pred, squared=False),
        'r2': r2_score(y_true, y_pred),
    }

def summarize_metrics(m):
    return f"MAE={m['mae']:.2f}, RMSE={m['rmse']:.2f}, R2={m['r2']:.3f}"

df_model = features_df.dropna(subset=['aqi'])
feature_cols = [c for c in (pollutant_cols + ['hour','day_of_week','month','latitude','longitude'] + [f'{p}_lag1' for p in pollutant_cols]) if c in df_model.columns]

train_df, test_df = train_test_split_time(df_model, time_col='timestamp', test_size=0.2)
X_train = train_df[feature_cols].values
y_train = train_df['aqi'].values
X_test = test_df[feature_cols].values
y_test = test_df['aqi'].values

baseline_model = Pipeline([('imputer', SimpleImputer(strategy='median')), ('model', LinearRegression())])
baseline_model.fit(X_train, y_train)
baseline_pred = baseline_model.predict(X_test)
baseline_metrics = regression_metrics(y_test, baseline_pred)
print('Baseline:', summarize_metrics(baseline_metrics))

param_grid = {
    'model__n_estimators': [200, 400],
    'model__max_depth': [None, 10, 20],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2],
}
pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')), ('model', RandomForestRegressor(random_state=SEED, n_jobs=-1))])
grid = GridSearchCV(pipeline, param_grid=param_grid, cv=TimeSeriesSplit(n_splits=3), scoring='neg_mean_absolute_error', n_jobs=-1)
grid.fit(X_train, y_train)
best_model = grid.best_estimator_
tree_pred = best_model.predict(X_test)
tree_metrics = regression_metrics(y_test, tree_pred)
print('Tree:', summarize_metrics(tree_metrics))
print('Best params:', grid.best_params_)

plt.figure(figsize=(6, 6))
plt.scatter(y_test, tree_pred, alpha=0.4)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual AQI')
plt.ylabel('Predicted AQI')
plt.title('Predicted vs Actual AQI')
plt.tight_layout()
plt.show()

model = best_model.named_steps['model']
if hasattr(model, 'feature_importances_'):
    importances = model.feature_importances_
    order = np.argsort(importances)[::-1]
    plt.figure(figsize=(8, 4))
    plt.bar(range(len(feature_cols)), importances[order])
    plt.xticks(range(len(feature_cols)), np.array(feature_cols)[order], rotation=45, ha='right')
    plt.title('Feature Importance')
    plt.tight_layout()
    plt.show()


## H. Evaluation & Interpretation

In [ ]:
test_results = test_df.copy()
test_results['pred'] = tree_pred
test_results['abs_error'] = (test_results['aqi'] - test_results['pred']).abs()

test_results['month'] = test_results['timestamp'].dt.month
mae_by_month = test_results.groupby('month')['abs_error'].mean()
plt.figure(figsize=(8, 4))
mae_by_month.plot(kind='bar')
plt.title('MAE by Month (Test Set)')
plt.xlabel('Month')
plt.ylabel('MAE')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
test_results['abs_error'].hist(bins=30)
plt.title('Absolute Error Distribution')
plt.xlabel('Absolute Error')
plt.ylabel('Count')
plt.tight_layout()
plt.show()


## I. Conclusions & Next Steps

- Data limitations: missing pollutants for many timestamps and locations.
- Improvements: add weather variables, refine aggregation windows, and apply sensor-level quality filters.
- Modeling: try gradient boosting and location-specific models.


In [ ]:
report_path = ROOT / 'reports' / 'summary.md'
rows_cleaned = len(df_model)
time_start = df_model['timestamp'].min().date()
time_end = df_model['timestamp'].max().date()
pct_missing_aqi = 100 * df_model['aqi'].isna().mean()

notes = 'Tree model performs better than baseline; remaining errors likely due to missing context.'

report = report_path.read_text()
report = report.replace('{{rows_cleaned}}', str(rows_cleaned))
report = report.replace('{{time_start}}', str(time_start))
report = report.replace('{{time_end}}', str(time_end))
report = report.replace('{{aggregation_window}}', 'Daily')
report = report.replace('{{pct_missing_aqi}}', '{:.2f}%'.format(pct_missing_aqi))
report = report.replace('{{best_model_name}}', 'RandomForestRegressor')
report = report.replace('{{mae}}', '{:.2f}'.format(tree_metrics['mae']))
report = report.replace('{{rmse}}', '{:.2f}'.format(tree_metrics['rmse']))
report = report.replace('{{r2}}', '{:.3f}'.format(tree_metrics['r2']))
report = report.replace('{{notes}}', notes)
report_path.write_text(report)

print(report)
